In [1]:
%pip install -q git+https://github.com/tensorflow/docs

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
%pip install tensorflow imutils opencv-python matplotlib imageio

     -------------------------------------- 437.9/437.9 MB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 11.2 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 9.9 MB/s eta 0:00:00
     -------------------------------------- 462.5/462.5 KB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 2.8/2.8 MB 11.7 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 KB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB ? eta 0:00:00
     ---------------------------------------- 13.9/13.9 MB 7.7 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 KB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.5/1.5 MB 11.5 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 9.7 MB/s eta 0:00:00
  Using cached wheel-0.37.1-py2.

In [1]:
#없어도 될듯..?
%wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
%tar xf ucf101_top5.tar.gz

UsageError: Line magic function `%wget` not found.


In [1]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [3]:
#데이터 확인
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 20
Total videos for testing: 6


,video_name,tag
16,16.avi,neck
15,15.avi,neck
4,4.avi,nose
6,6.avi,nose
1,1.avi,nose
11,11.avi,neck
12,12.avi,neck
17,17.avi,neck
3,3.avi,nose
2,2.avi,nose


In [4]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [5]:
#비디오 불러와서 전처리
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [6]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [7]:
#카테고리 찾기
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['neck', 'nose']


In [8]:
#추후에 데이터 파일안에 정리
sample = [1,2,3,4]
var2=[]

for i in sample:
    var2 = str(sample[i-1])+".avi"

var2

'4.avi'

In [9]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    print(video_paths)
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels

In [10]:
train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

['0.avi', '1.avi', '2.avi', '3.avi', '4.avi', '5.avi', '6.avi', '7.avi', '8.avi', '9.avi', '10.avi', '11.avi', '12.avi', '13.avi', '14.avi', '15.avi', '16.avi', '17.avi', '18.avi', '19.avi']
['20.avi', '21.avi', '22.avi', '23.avi', '24.avi', '25.avi']
Frame features in train set: (20, 20, 2048)
Frame masks in train set: (20, 20)


In [14]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = tf.keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    
    x = tf.keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = tf.keras.layers.GRU(8)(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(8, activation="relu")(x)
    output = tf.keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = tf.keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/train_data_result"
    #checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, save_weights_only=False, save_best_only=True, verbose=1)
    #cb_earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss",mode="auto",verbose=1,patience=2)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        #callbacks = [cb_earlystop, checkpoint],
    )
#test accuracy 확인하기
    #seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    #print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model 

    

_, sequence_model = run_experiment()

Epoch 1/10
1/1 [==============================] - 7s 7s/step - loss: 0.6931 - accuracy: 0.2857 - val_loss: 0.6941 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 45ms/step - loss: 0.6927 - accuracy: 0.7143 - val_loss: 0.6951 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 53ms/step - loss: 0.6923 - accuracy: 0.7143 - val_loss: 0.6962 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 50ms/step - loss: 0.6919 - accuracy: 0.7143 - val_loss: 0.6972 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 50ms/step - loss: 0.6914 - accuracy: 0.7143 - val_loss: 0.6982 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 45ms/step - loss: 0.6910 - accuracy: 0.7143 - val_loss: 0.6992 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 46ms/step - loss: 0.6906 - accuracy: 0.7143 - val_loss: 0.7002 - val_accuracy: 0.0000e+00
E